# Problem formulation

1. Define decision variables
2. Define constraints (on the decision variables)
3. Define objective function

## Global decision variables - FSI

Inputs required:

- maximum building extents in 3D (in the form of a .obj to be provided by user)
- the current occupation lattice

NB THESE ARE ACTUALLY OBJECTIVE FUNCTIONS

In [3]:
# first we define the decision variables for the programmatic minimum requirement of the entire project (manifesting itself as FSI)
# then we find the solar yield of the entire project, this includes solar heat gain and PV potential as aggregated values (manifesting itself as collisisons per m2)

import trimesh as tm
import os
import pyvista as pv
import numpy as np
import topogenesis as tg

lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # the current occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height, USER INPUT)

base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint
env_all_vox = occ_lattice.flatten() # flattened lattice of all possible positions, True is occupied, False is unoccupied
FSI_min = 2.0 # current goal for the FSI, can be a variable

# FSI DO: maybe it is faster to take footprint outside of the equation so it does not recompute the footprint everytime?

def FSI(footprint, occupationlattice): # calculate FSI based on occupation lattice and mesh plot
    vox_size = occupationlattice.unit[0]
    m2_plot = footprint.area/2 # area calculates both sides of the flattened mesh, we need only 1
    cell_count = [i for i in occupationlattice if i != False ]
    m2_total = vox_size**2*len(cell_count)
    FSI = m2_total/m2_plot
    return FSI

def FSI_fast(floors, floor_percent=0.8): # calculate FSI based on floorcount, plot coverage of the floors
    m2_floor = floor_percent
    FSI = m2_floor*floors
    #FSI = m2_total/m2_plot
    return FSI

# FSI(base,env_all_vox)
FSI_fast(3)

2.4000000000000004

In [4]:
import trimesh as tm
import os
import pyvista as pv
import numpy as np
import topogenesis as tg

In [5]:

lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # the current occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height, USER INPUT)

In [6]:
np.arange(9)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

## Global decision variables - Aggregated Solar Score (ASS) (working title) (PV and heat gain)

Inputs required:

- Building mass (mesh or .obj) OR floorcount + roof area
- Coordinates OR solar vectors (prepared) using Ladybug plugin
- DO: check values for constants, check if it is needed iteratively or only global


In [7]:
# this includes a. the PV potential (calculated by roof area over total floor area)
# and b. the solar heat gain over the year (calculated by G-value/SHGC, window/wall ratio, solar heat gain coefficient of opaque components)

# PV potential - how much roof area per m2 is available for installing PV

def PV_fast(floors): # this gives ESTIMATED RATIO between floor area and PV area. Needs to be recomputed at the end of the local scale (or continuously calculated) DO: find out
    PV = 1/floors
    return PV

# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

r = 0.15 # yield/efficiency estimate
H = 1050 # to be updated, currently wikipedia estimate. Slope, tilt, orientation may change values. kWh.y so maybe *365???
PR = 0.75 # default value, estimate

def PV(occupationlattice): # an accurate value (kWh estimate) of the PV potential DO: change to only check for the added voxels of the current iteration?
    vox_size = occupationlattice.unit[0]
    cell_count = [ i for i in occupationlattice if i != False ] # change to only include voxels on the 'roof'
    A = len(cell_count)*vox_size # update to actual roof area
    E = A*r*H*PR # the estimated solar yield in kWh for the year
    return E

PV(env_all_vox)
# solar heat gain (windows only), include opaque surfaces?



# Aggregated score



652050.0

## Local cost/objective function - Energy usage (minimize)

inputs required:

- current floor area (per building unit type)
- constant value (per building unit type)

In [8]:
# skip for now, maybe skip permanently? --> too dependant on building (material) quality and not only on massing/envelope

## Local cost/objective function - daylighting (threshold)

inputs required:

- Current building unit (type)
- Surrounding voxels
- Solar path